# About the competitions

### Experiment with the largest-ever self-driving Prediction Dataset to build motion prediction model.

# Prediction Dataset sample

![](http://https://self-driving.lyft.com/wp-content/uploads/2020/06/motion_dataset_2-1.png)

![](https://self-driving.lyft.com/wp-content/uploads/2020/06/motion_dataset_2-1.png)

# What is autonomous vehicle ?
A self-driving car, also known as an autonomous vehicle (AV), connected and autonomous vehicle (CAV), driverless car, robo-car, or robotic car,(automated vehicles and fully automated vehicles in the European Union) is a vehicle that is capable of sensing its environment and moving safely with little or no human input.

Self-driving cars combine a variety of sensors to perceive their surroundings, such as radar, lidar, sonar, GPS, odometry and inertial measurement units. Advanced control systems interpret sensory information to identify appropriate navigation paths, as well as obstacles and relevant signage


# Example prediction solution

![](https://self-driving.lyft.com/wp-content/uploads/2020/06/diagram-prediction-1.jpg)

# Scene Visualisation

Code is taken from: 

https://github.com/lyft/l5kit

https://github.com/lyft/l5kit/blob/master/examples/visualisation/visualise_data.ipynb


In [ ]:
!pip install --upgrade pip
!pip install pymap3d==2.1.0
!pip install -U l5kit

# Imports

In [ ]:
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

from l5kit.data import ChunkedDataset, LocalDataManager
from l5kit.dataset import EgoDataset, AgentDataset

from l5kit.rasterization import build_rasterizer
from l5kit.configs import load_config_data
from l5kit.visualization import draw_trajectory, TARGET_POINTS_COLOR
from l5kit.geometry import transform_points
from tqdm import tqdm
from collections import Counter
from l5kit.data import PERCEPTION_LABELS
from prettytable import PrettyTable

import os

from matplotlib import animation, rc
from IPython.display import HTML

rc('animation', html='jshtml')

In [ ]:
def animate_solution(images):

    def animate(i):
        im.set_data(images[i])
 
    fig, ax = plt.subplots()
    im = ax.imshow(images[0])
    
    return animation.FuncAnimation(fig, animate, frames=len(images), interval=60)

In [ ]:
# set env variable for data
os.environ["L5KIT_DATA_FOLDER"] = "../input/lyft-motion-prediction-autonomous-vehicles"
# get config
cfg = load_config_data("../input/lyft-config-files/visualisation_config.yaml")
print(cfg)

# Load the data

In [ ]:
dm = LocalDataManager()
dataset_path = dm.require(cfg["val_data_loader"]["key"])
zarr_dataset = ChunkedDataset(dataset_path)
zarr_dataset.open()
print(zarr_dataset)

# DATA FORMAT


 The dataset is provided in zarr format. The zarr files are flat, compact, and highly performant for loading. To read the dataset please use our new Python software kit.
 The dataset consists of frames and agent states. A frame is a snapshot in time which consists of ego pose, time, and multiple agent states. Each agent state describes the position, orientation, bounds, and type.

# EXAMPLE

In [ ]:
import zarr

In [ ]:
train_zarr = zarr.open("../input/lyft-motion-prediction-autonomous-vehicles/scenes/train.zarr")

print(type(train_zarr))


In [ ]:
train_zarr.info

### The 2020 Lyft competition dataset is stored in four structured arrays: scenes, frames, agents and tl_faces.
 *Note: in the following all _interval fields assume that information is stored consecutively in the arrays. This means that if frame_index_interval for scene_0 is (0, 100), frames from scene_1 will start from index 100 in the frames array.*

 Reading from a zarr array is as easy as slicing from it like you would any numpy array. The return value is an ordinary numpy array. Zarr takes care of determining which chunks to read from.

In [ ]:
train_zarr.agents[:5]

# Data Overview

In [ ]:
agents = zarr_dataset.agents
probabilities = agents["label_probabilities"]
labels_indexes = np.argmax(probabilities, axis=1)
counts = []
for idx_label, label in enumerate(PERCEPTION_LABELS):
    counts.append(np.sum(labels_indexes == idx_label))
    
table = PrettyTable(field_names=["label", "counts"])
for count, label in zip(counts, PERCEPTION_LABELS):
    table.add_row([label, count])
print(table)

In [ ]:
rast = build_rasterizer(cfg, dm)
dataset = EgoDataset(cfg, zarr_dataset, rast)

# Autonomous Vehicle with Trajectory

In [ ]:
data = dataset[80]

im = data["image"].transpose(1, 2, 0)
im = dataset.rasterizer.to_rgb(im)
target_positions_pixels = transform_points(data["target_positions"] + data["centroid"][:2], data["world_to_image"])
draw_trajectory(im, target_positions_pixels, data["target_yaws"], TARGET_POINTS_COLOR)
figsize = plt.subplots(figsize = (10,10))
plt.title('Autonomous Vehicle with Trajectory',fontsize=20)

plt.imshow(im[::-1])
plt.show()

# Satellite View

In [ ]:
cfg["raster_params"]["map_type"] = "py_satellite"
rast = build_rasterizer(cfg, dm)
dataset = EgoDataset(cfg, zarr_dataset, rast)
data = dataset[80]

im = data["image"].transpose(1, 2, 0)
im = dataset.rasterizer.to_rgb(im)
target_positions_pixels = transform_points(data["target_positions"] + data["centroid"][:2], data["world_to_image"])
draw_trajectory(im, target_positions_pixels, data["target_yaws"], TARGET_POINTS_COLOR)
figsize = plt.subplots(figsize = (10,10))
plt.title('Satellite View',fontsize=20)
plt.imshow(im[::-1])
plt.show()

In [ ]:
dataset = AgentDataset(cfg, zarr_dataset, rast)
data = dataset[80]

im = data["image"].transpose(1, 2, 0)
im = dataset.rasterizer.to_rgb(im)
target_positions_pixels = transform_points(data["target_positions"] + data["centroid"][:2], data["world_to_image"])
draw_trajectory(im, target_positions_pixels, data["target_yaws"], TARGET_POINTS_COLOR)

fig = plt.subplots(figsize=(10,10))
plt.imshow(im[::-1])
plt.show()

 Green represents the autonomous vehicle (AV), and blue is primarily all the other cars/vehicles/exogenous factors we need to predict for.
 If we are able to accurately predict the path the vehicles go through, it will make it easier for an AV to compute its trajectory on the fly.

#  Entire Scene

## Semantic

What is semantic map?



In [ ]:
from IPython.display import display, clear_output
import PIL
 
cfg["raster_params"]["map_type"] = "py_semantic"
rast = build_rasterizer(cfg, dm)
dataset = EgoDataset(cfg, zarr_dataset, rast)
scene_idx = 34
indexes = dataset.get_scene_indices(scene_idx)
images = []

for idx in indexes:
    
    data = dataset[idx]
    im = data["image"].transpose(1, 2, 0)
    im = dataset.rasterizer.to_rgb(im)
    target_positions_pixels = transform_points(data["target_positions"] + data["centroid"][:2], data["world_to_image"])
    center_in_pixels = np.asarray(cfg["raster_params"]["ego_center"]) * cfg["raster_params"]["raster_size"]
    draw_trajectory(im, target_positions_pixels, data["target_yaws"], TARGET_POINTS_COLOR)
    clear_output(wait=True)
    images.append(PIL.Image.fromarray(im[::-1]))

In [ ]:
anim = animate_solution(images)
HTML(anim.to_jshtml())

## Satellite

In [ ]:
from IPython.display import display, clear_output
import PIL
 
cfg["raster_params"]["map_type"] = "py_satellite"
rast = build_rasterizer(cfg, dm)
dataset = EgoDataset(cfg, zarr_dataset, rast)
scene_idx = 34
indexes = dataset.get_scene_indices(scene_idx)
images = []


for i, idx in enumerate(indexes):
    data = dataset[idx]
    im = data["image"].transpose(1, 2, 0)
    im = dataset.rasterizer.to_rgb(im)
    target_positions_pixels = transform_points(data["target_positions"] + data["centroid"][:2], data["world_to_image"])
    center_in_pixels = np.asarray(cfg["raster_params"]["ego_center"]) * cfg["raster_params"]["raster_size"]
    draw_trajectory(im, target_positions_pixels, data["target_yaws"], TARGET_POINTS_COLOR)
    clear_output(wait=True)
    images.append(PIL.Image.fromarray(im[::-1]))

In [ ]:
anim = animate_solution(images)
HTML(anim.to_jshtml())

# Sample Submission

In [ ]:
DIR_INPUT = "/kaggle/input/lyft-motion-prediction-autonomous-vehicles"

SINGLE_MODE_SUBMISSION = f"{DIR_INPUT}/single_mode_sample_submission.csv"
MULTI_MODE_SUBMISSION = f"{DIR_INPUT}/multi_mode_sample_submission.csv"

In [ ]:
# Single mode
sample_submission = pd.read_csv(SINGLE_MODE_SUBMISSION)

# Multi mode
# sample_submission = pd.read_csv(MULTI_MODE_SUBMISSION)

In [ ]:
sample_submission.head()


In [ ]:
sample_submission.to_csv("submission.csv", index=False)